In [ ]:
# ✅ 필요한 라이브러리 불러오기
from datasets import load_dataset
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
print("✅ 라이브러리 불러오기 완료")

In [ ]:
# ✅ 데이터가 존재하는지 확인 후, 없으면 다운로드 및 저장
data_path = Path('./data/student_performance.csv')
if not data_path.exists():
    print("🔽 데이터 다운로드 중...")
    dataset = load_dataset("neuralsorcerer/student-performance")
    df = dataset['train'].to_pandas()
    data_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(data_path, index=False)
    print("✅ 데이터 저장 완료")
else:
    print("📂 저장된 데이터 불러오기")
    df = pd.read_csv(data_path)

In [ ]:
# ✅ 필요한 컬럼만 선택하고 결측치 제거
cols = ['Gender', 'ParentalEducation', 'TestScore_Math', 'TestScore_Reading', 'TestScore_Science']
df = df[cols].dropna()
print(f"📏 정제된 데이터 크기: {df.shape}")

In [ ]:
# ✅ 데이터 분할 및 저장, 이미 존재하면 불러오기
train_path = Path('./data/train.csv')
test_path = Path('./data/test.csv')
if not train_path.exists() or not test_path.exists():
    print("🔀 데이터 분할 중...")
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)
    print("✅ 분할 데이터 저장 완료")
else:
    print("📂 저장된 train/test 데이터 불러오기")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

In [ ]:
# ✅ 시각화: 과목별 점수 분포 확인
sns.pairplot(train_df[['TestScore_Math', 'TestScore_Reading', 'TestScore_Science']])
plt.suptitle("📊 과목별 점수 분포 및 상관관계", y=1.02)
plt.show()

In [ ]:
# ✅ 회귀 분석: 과학 점수를 수학/읽기 점수로 예측
X_train = train_df[['TestScore_Math', 'TestScore_Reading']]
y_train = train_df['TestScore_Science']
X_test = test_df[['TestScore_Math', 'TestScore_Reading']]
y_test = test_df['TestScore_Science']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.2f}")

In [ ]:
# ✅ 예측 결과 시각화
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.xlabel("Actual Science Scores")
plt.ylabel("Predicted Science Scores")
plt.title("🎯 예측 vs 실제: 과학 점수")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.grid(True)
plt.show()